# SW 02 Exercises Solutions

## Part 1: Generate 2-3 synthetic time series with known components and review how well components are extracted

Deif

## Part 2: Generate and interpret ACF plots

Deif

## Part 3: Reimplement classical decomposition

Meiko

## Part 3: Extend lecture 1 exercise with ACF plots and time series decomposition

Nevin